# AGNQSO Summary catalogue generator

This notebook demonstrates the catalog generation.

All functions and final wrapper script to live in py/ * .py.

If you are on NERSC please select 'DESI main' as your kernel.

Notebook direct contirbutions:

* Alexander, D (Univ. of Durham, Durham, UK) (VI merging done by DA)
* Alfarsy, R (Univ. of Portsmouth, Portsmouth, UK)
* Canning, B (Univ. of Portsmouth, Portsmouth, UK)
* Chaussidon, E (CEA Saclay, Paris, France) (QSO catalogs generated by EC et al.)
* Juneau, S (NOIRLab, Arizona, USA)
* Mezcua, M (Institut de Ciències de l'Espai, Barcelona, Spain)
* Moustakas, J (Siena College, New York, USA) (FastSpecFit catalogues by JM)
* Pucha, R (Univ. of Arizona, Arizona, USA) 

## Docs:
Readme:
Wiki: 
Github: 

Directions for VACs: https://desi.lbl.gov/trac/wiki/Pipeline/Releases/EDR/Planning/ValueAdded

Our VAC directory: /global/cfs/cdirs/desi/science/gqp/agncatalog/  

## Imports

In [1]:
# General imports
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import os.path
#import yaml

# Import Astropy libraries - useful for many astronomy related function
from astropy.table import Table, join, Column
from astropy.io import fits
import fitsio

# DESI modules
from desispec.zcatalog import find_primary_spectra, create_summary_catalog  # at NERSC needs DESI master
from desitarget.targetmask import desi_mask, bgs_mask, scnd_mask      # For the main survey
#from desiutil.bitmask import BitMask

# GQP_CODE
import sys
sys.path.append("../py/")
import set_agn_masksDESI

#https://www.legacysurvey.org/viewer?ra=10.1572&dec=-0.3316&layer=ls-dr9&zoom=16

In [2]:
## Making the matplotlib plots look nicer - from SJ
settings = {
    'font.size':16,
    'axes.linewidth':2.0,
    'xtick.major.size':6.0,
    'xtick.minor.size':4.0,
    'xtick.major.width':2.0,
    'xtick.minor.width':1.,
    'xtick.direction':'in', 
    'xtick.minor.visible':True,
    'xtick.top':True,
    'ytick.major.size':6.0,
    'ytick.minor.size':4.0,
    'ytick.major.width':2.0,
    'ytick.minor.width':1.,
    'ytick.direction':'in', 
    'ytick.minor.visible':True,
    'ytick.right':True
}

plt.rcParams.update(**settings)

# Workflow

Deveopment code in: /global/homes/b/bcanning/AGNQSO_summary_catalog/

1. Read Edmond threshold catalog
   * Drop ['COADD_FIBERSTATUS']==0
   * OBJ_TYPE = TGT
   * low-z star cut
   * zwarn cut
   * low-z galaxy fragments
   * frac_flux numbers but John would like maskbits
   
   
   * Note: Fuji has defaut 0.95 c_thresh for all targets
     - If BGS target (not QSO, not ELG, not LRG): RR SPECTYPE=QSO and (QN C_LINE_BEST>0.6 or MgII)
     - If ELG (not QSO): RR SPECTYPE=QSO and QN C_LINE_BEST>0.6
     - If QSO threshold 'QN_C_LINE_BEST' > 0.95 (Default)

2. Generate (or read) summary catalog
   * Join to add 'TSNR2_LRG','SV_NSPEC','SV_PRIMARY','ZCAT_NSPEC','ZCAT_PRIMARY'
   * Generate 'QN_C_LINE_BEST' and 'QN_C_LINE_SECOND_BEST'

3. Join FastSpecFit v2 - specific columns

4. Read yaml file

5. Set QSO_MASKBITS part of AGN_MASKBITS

6. Set BPT bits 
   * Update AGN_MASKBITS 

7. Join multiwave survey

8. Write catalog

## Basic Info (edit for choosing data release)

In [3]:
GQP_AGNcat_dir='/global/cfs/cdirs/desi/science/gqp/agncatalog/'
dir_for_cat=GQP_AGNcat_dir+'catalog/'
#dir_for_files='py/'
dir_for_tmp=GQP_AGNcat_dir+'tmp/'

filetype = 'healpix'
specgroup_type = 'zpix'

# Which spectroscopic release
specprod = 'fuji'
#specprod = 'guadalupe'
#specprod = 'iron'

#### OTHER FILES NEEDED 
# Edmonds catalogue from QSO maker keeping all columns
path_qsom = f'/global/cfs/cdirs/desi/users/edmondc/QSO_catalog/{specprod}/'  #NERSC
file_qsom = path_qsom+f'QSO_cat_{specprod}_healpix_all_targets_v2.fits'

# FastSpecFit file
fast_dir = f'/global/cfs/cdirs/desi/spectro/fastspecfit/{specprod}/v3.2/catalogs/'
fastspec_file = fast_dir+f'fastspec-{specprod}.fits'
#fastphot_file = fast_dir+f'fastphot-{specprod}.fits'

In [4]:
# Main identifiers for Joins
keys_for_join=['TARGETID','SURVEY','PROGRAM']

In [5]:
# Data model
#
# Question about whether to keep: 
#  - LS_ID - this one from FSF
#  - other Tractor cols (MORPHTYPE, MASKBITS, PHOTSYS)
#  - EBV_1 (where does this one come from? from joining two tables with EBV??)
#  - FIBERFLUX* and FIBERTOTFLUX*
#  + replaced HPXPIXEL with HEALPIX (updated name); added BGS targeting cols to find BGS_WISE
#  + mjd information min, mean, max, zcat and primary stuff: https://github.com/desihub/desispec/blob/master/py/desispec/zcatalog.py
#  + removed QSO_MASKBITS as repeated in AGN_MASKBITS
#  + where does the Z come from - FSF or QSO maker - make it FSF!
#
#final_cols=['TARGETID','SURVEY','PROGRAM','HEALPIX','Z','ZERR','ZWARN','SPECTYPE','COADD_FIBERSTATUS','TARGET_RA','TARGET_DEC',\
#            'MORPHTYPE','EBV_1','MASKBITS',\
#            'DESI_TARGET','SCND_TARGET','BGS_TARGET','COADD_NUMEXP','COADD_EXPTIME','CMX_TARGET',\
#            'SV1_DESI_TARGET','SV2_DESI_TARGET','SV3_DESI_TARGET','SV1_BGS_TARGET','SV2_BGS_TARGET','SV3_BGS_TARGET',\
#            'SV1_SCND_TARGET','SV2_SCND_TARGET','SV3_SCND_TARGET',\
#            'TSNR2_LYA','TSNR2_QSO','TSNR2_LRG',\
#            'DELTA_CHI2_MGII','A_MGII','SIGMA_MGII','B_MGII','VAR_A_MGII','VAR_SIGMA_MGII','VAR_B_MGII',\
#            'Z_RR','Z_QN','C_LYA','C_CIV','C_CIII','C_MgII','C_Hbeta','C_Halpha','Z_LYA','Z_CIV','Z_CIII','Z_MgII','Z_Hbeta','Z_Halpha',\
#            'SV_NSPEC','SV_PRIMARY','ZCAT_NSPEC','ZCAT_PRIMARY',\
#            'QN_C_LINE_BEST','QN_C_LINE_SECOND_BEST','QSO_MASKBITS','AGN_MASKBITS','AGN_TYPE',\
#            'PHOTSYS','LS_ID','FIBERFLUX_G','FIBERFLUX_R','FIBERFLUX_Z','FIBERTOTFLUX_G','FIBERTOTFLUX_R','FIBERTOTFLUX_Z'
#.           'MIN_MJD','MEAN_MJD','MAX_MJD']

## SJ: should we keep QN_C_LINE_BEST?
## SJ: Do we need 'MEAN_MJD'?
final_cols=['TARGETID', 'SURVEY', 'PROGRAM', 'HEALPIX', 'Z', 'ZERR', 'ZWARN', 'SPECTYPE', \
            'AGN_MASKBITS', 'OPT_UV_TYPE', 'IR_TYPE', 'COADD_FIBERSTATUS', \
            'TARGET_RA', 'TARGET_DEC', 'LS_ID', 'MIN_MJD','MEAN_MJD','MAX_MJD','COADD_NUMEXP', 'COADD_EXPTIME', \
            'SV_PRIMARY','ZCAT_PRIMARY','DESI_TARGET', 'SCND_TARGET', 'BGS_TARGET', 'CMX_TARGET', \
            'SV1_DESI_TARGET', 'SV2_DESI_TARGET', 'SV3_DESI_TARGET', 'SV1_BGS_TARGET', 'SV2_BGS_TARGET', 'SV3_BGS_TARGET', \
            'SV1_SCND_TARGET', 'SV2_SCND_TARGET', 'SV3_SCND_TARGET']

## Column for "convenience" extension with values for plotting
ext2_cols=['TARGETID','SURVEY','PROGRAM','LOGMSTAR',\
           'FLUX_W1','FLUX_W2','FLUX_W3',\
           'FLUX_IVAR_W1','FLUX_IVAR_W2','FLUX_IVAR_W3',\
           'CIV_1549_FLUX','CIV_1549_FLUX_IVAR', 'CIV_1549_SIGMA',\
           'MGII_2796_FLUX','MGII_2796_FLUX_IVAR','MGII_2796_SIGMA',\
           'MGII_2803_FLUX','MGII_2803_FLUX_IVAR', 'MGII_2803_SIGMA',\
           'OII_3726_FLUX','OII_3726_FLUX_IVAR','OII_3726_EW','OII_3726_EW_IVAR',\
           'OII_3729_FLUX','OII_3729_FLUX_IVAR','OII_3729_EW','OII_3729_EW_IVAR',\
           'NEV_3426_FLUX','NEV_3426_FLUX_IVAR',\
           'HEII_4686_FLUX','HEII_4686_FLUX_IVAR',\
           'HBETA_EW','HBETA_EW_IVAR','HBETA_FLUX','HBETA_FLUX_IVAR',\
           'HBETA_BROAD_FLUX', 'HBETA_BROAD_FLUX_IVAR', 'HBETA_BROAD_SIGMA','HBETA_BROAD_CHI2',\
           'OIII_5007_FLUX','OIII_5007_FLUX_IVAR','OIII_5007_SIGMA',\
           'OI_6300_FLUX','OI_6300_FLUX_IVAR',\
           'HALPHA_EW', 'HALPHA_FLUX','HALPHA_FLUX_IVAR','HALPHA_SIGMA', \
           'HALPHA_BROAD_FLUX','HALPHA_BROAD_FLUX_IVAR','HALPHA_BROAD_VSHIFT','HALPHA_BROAD_SIGMA',\
           'NII_6584_FLUX','NII_6584_FLUX_IVAR',\
           'SII_6716_FLUX','SII_6716_FLUX_IVAR',\
           'SII_6731_FLUX','SII_6731_FLUX_IVAR']

# Longer list (all that we use?)
# ext2_cols=['TARGETID','SURVEY','PROGRAM','LOGMSTAR',\
#            'CIV_1549_FLUX','CIV_1549_FLUX_IVAR', 'CIV_1549_SIGMA',\
#            'MGII_2796_FLUX','MGII_2796_FLUX_IVAR','MGII_2796_SIGMA',\
#            'MGII_2803_FLUX','MGII_2803_FLUX_IVAR', 'MGII_2803_SIGMA',\
#            'OII_3726_FLUX','OII_3726_FLUX_IVAR','OII_3726_EW',\
#            'NEV_3426_FLUX','NEV_3426_FLUX_IVAR',\
#            'HEII_4686_FLUX','HEII_4686_FLUX_IVAR',\
#            'HBETA_EW','HBETA_FLUX','HBETA_FLUX_IVAR',\
#            'HBETA_BROAD_FLUX', 'HBETA_BROAD_FLUX_IVAR', 'HBETA_BROAD_VSHIFT','HBETA_BROAD_SIGMA','HBETA_BROAD_CHI2',\
#            'OIII_5007_FLUX','OIII_5007_FLUX_IVAR','OIII_5007_SIGMA',\
#            'OI_6300_FLUX','OI_6300_FLUX_IVAR',\
#            'HALPHA_EW', 'HALPHA_FLUX','HALPHA_FLUX_IVAR','HALPHA_SIGMA', \
#            'HALPHA_BROAD_FLUX','HALPHA_BROAD_FLUX_IVAR','HALPHA_BROAD_VSHIFT','HALPHA_BROAD_SIGMA',\
#            'NII_6584_FLUX','NII_6584_FLUX_IVAR',\
#            'SII_6716_FLUX','SII_6716_FLUX_IVAR',\
#            'SII_6731_FLUX','SII_6731_FLUX_IVAR']

print(final_cols)

['TARGETID', 'SURVEY', 'PROGRAM', 'HEALPIX', 'Z', 'ZERR', 'ZWARN', 'SPECTYPE', 'AGN_MASKBITS', 'OPT_UV_TYPE', 'IR_TYPE', 'COADD_FIBERSTATUS', 'TARGET_RA', 'TARGET_DEC', 'LS_ID', 'MIN_MJD', 'MEAN_MJD', 'MAX_MJD', 'COADD_NUMEXP', 'COADD_EXPTIME', 'SV_PRIMARY', 'ZCAT_PRIMARY', 'DESI_TARGET', 'SCND_TARGET', 'BGS_TARGET', 'CMX_TARGET', 'SV1_DESI_TARGET', 'SV2_DESI_TARGET', 'SV3_DESI_TARGET', 'SV1_BGS_TARGET', 'SV2_BGS_TARGET', 'SV3_BGS_TARGET', 'SV1_SCND_TARGET', 'SV2_SCND_TARGET', 'SV3_SCND_TARGET']


# Define cuts that might be wanted

The below function should be run on the final joined catalogs as not all keywords exist otherwise.

We are not making any cuts currnetly. 

In [6]:
# def cut_fiberstatus(T):
#     ''' 
#     keep only objects with 'COADD_FIBERSTATUS' == 0
#     '''
#     keep = (T['COADD_FIBERSTATUS']==0)
#     return T[keep]

# def cut_npixels(T):
#     ''' 
#     keep only objects with 'NPIXELS' > 0 (signifying they have a coadded spectrum)
#     '''
#     keep = (T['NPIXELS']>0)
#     return T[keep]

# def cut_zwarn(T):
#     ''' 
#     keep only objects with                  (zb['ZWARN'] & ZWarningMask.NODATA == 0))[0]
# https://fastspecfit.readthedocs.io/en/latest/vacs.html#sample-selection
#     '''
#     keep = (T['ZWARN'] & ZWarningMask.NODATA ==0)  # might not be written correctly...
#     return T[keep]

def cut_objtype(T):
    ''' 
    keep only objects with 'OBJTYPE' == 'TGT'
    '''
    keep = (T['OBJTYPE']=='TGT')
    return T[keep]

def cut_lowz_star(T):
    ''' 
    keep only objects with redshift greater than 0.001
    '''
    keep = (T['Z']>0.001)
    return T[keep]

# def cut_lowz_galfragments(T):
#     ''' 
#     keep only objects with 
#     '''
#     keep = (T['']==)
#     return T[keep]    

### cuts added by EC but leaving in here for discussion with SJ

# #### Notes/Questions
# - How to treat objects that might have more than one target type?
# - Correct bump at z~3.7:
# ```
#     sel_pb_redshift = (QSO_cat['Z'] > 3.65) & ((QSO_cat['C_LYA']<0.95) | (QSO_cat['C_CIV']<0.95))
# ```

In [7]:
#test1=cut_objtype(T_qsom)
#test2=cut_lowz_star(T_qsom)
#print(len(test2))

## 1. QSO-maker Cat

In [8]:
%%time
T_qsom = Table(fitsio.read(file_qsom)) 

CPU times: user 10 s, sys: 3.85 s, total: 13.8 s
Wall time: 14 s


In [9]:
print(len(T_qsom))

## Remove stars with a low redshift cut
T_qsom = cut_lowz_star(T_qsom)

print(len(T_qsom))
print(T_qsom.columns)

2847435
1827985
<TableColumns names=('TARGETID','Z','ZERR','ZWARN','SPECTYPE','COADD_FIBERSTATUS','TARGET_RA','TARGET_DEC','MORPHTYPE','EBV','FLUX_G','FLUX_R','FLUX_Z','FLUX_W1','FLUX_W2','FLUX_IVAR_G','FLUX_IVAR_R','FLUX_IVAR_Z','FLUX_IVAR_W1','FLUX_IVAR_W2','MASKBITS','DESI_TARGET','SCND_TARGET','COADD_NUMEXP','COADD_EXPTIME','CMX_TARGET','SV1_DESI_TARGET','SV2_DESI_TARGET','SV3_DESI_TARGET','SV1_SCND_TARGET','SV2_SCND_TARGET','SV3_SCND_TARGET','TSNR2_LYA','TSNR2_QSO','DELTA_CHI2_MGII','A_MGII','SIGMA_MGII','B_MGII','VAR_A_MGII','VAR_SIGMA_MGII','VAR_B_MGII','Z_RR','Z_QN','C_LYA','C_CIV','C_CIII','C_MgII','C_Hbeta','C_Halpha','Z_LYA','Z_CIV','Z_CIII','Z_MgII','Z_Hbeta','Z_Halpha','QSO_MASKBITS','HPXPIXEL','SURVEY','PROGRAM')>


In [10]:
## SJ: will exclude the targeting cols because we'll add them from the zcat VAC instead 
#qsom_cols=['TARGETID','Z','ZERR','ZWARN','SPECTYPE','COADD_FIBERSTATUS','TARGET_RA','TARGET_DEC','MORPHTYPE','EBV','MASKBITS','DESI_TARGET','SCND_TARGET','COADD_NUMEXP','COADD_EXPTIME','CMX_TARGET','SV1_DESI_TARGET','SV2_DESI_TARGET','SV3_DESI_TARGET','SV1_SCND_TARGET','SV2_SCND_TARGET','SV3_SCND_TARGET','TSNR2_LYA','TSNR2_QSO','DELTA_CHI2_MGII','A_MGII','SIGMA_MGII','B_MGII','VAR_A_MGII','VAR_SIGMA_MGII','VAR_B_MGII','Z_RR','Z_QN','C_LYA','C_CIV','C_CIII','C_MgII','C_Hbeta','C_Halpha','Z_LYA','Z_CIV','Z_CIII','Z_MgII','Z_Hbeta','Z_Halpha','QSO_MASKBITS','HPXPIXEL','SURVEY','PROGRAM']
qsom_cols=['TARGETID','Z','ZERR','ZWARN','SPECTYPE','COADD_FIBERSTATUS','TARGET_RA','TARGET_DEC','MORPHTYPE','MASKBITS',\
           'COADD_NUMEXP','COADD_EXPTIME','TSNR2_LYA','TSNR2_QSO',\
           'DELTA_CHI2_MGII','A_MGII','SIGMA_MGII','B_MGII','VAR_A_MGII','VAR_SIGMA_MGII','VAR_B_MGII',\
           'Z_RR','Z_QN','C_LYA','C_CIV','C_CIII','C_MgII','C_Hbeta','C_Halpha','Z_LYA','Z_CIV','Z_CIII','Z_MgII','Z_Hbeta','Z_Halpha',\
           'QSO_MASKBITS','SURVEY','PROGRAM']

print(qsom_cols)

['TARGETID', 'Z', 'ZERR', 'ZWARN', 'SPECTYPE', 'COADD_FIBERSTATUS', 'TARGET_RA', 'TARGET_DEC', 'MORPHTYPE', 'MASKBITS', 'COADD_NUMEXP', 'COADD_EXPTIME', 'TSNR2_LYA', 'TSNR2_QSO', 'DELTA_CHI2_MGII', 'A_MGII', 'SIGMA_MGII', 'B_MGII', 'VAR_A_MGII', 'VAR_SIGMA_MGII', 'VAR_B_MGII', 'Z_RR', 'Z_QN', 'C_LYA', 'C_CIV', 'C_CIII', 'C_MgII', 'C_Hbeta', 'C_Halpha', 'Z_LYA', 'Z_CIV', 'Z_CIII', 'Z_MgII', 'Z_Hbeta', 'Z_Halpha', 'QSO_MASKBITS', 'SURVEY', 'PROGRAM']


In [11]:
T_qsom=T_qsom[qsom_cols]

In [12]:
print(len(T_qsom))

1827985


## 2. Use the Redshift Summary (zcat) VAC

The Redshift Summary Catalog VAC supersedes the original redshift catalogs for Fuji (EDR) as described [here](https://data.desi.lbl.gov/doc/releases/edr/vac/zcat/).

In [13]:
#file_zpix_sum_cat=dir_for_tmp+'zpix-'+specprod+'-summary.fits'
# Using the public EDR version of the zcat VAC
file_zpix_sum_cat = '/global/cfs/cdirs/desi/public/edr/vac/edr/zcat/fuji/v1.0/zall-pix-edr-vac.fits'

In [14]:
# from https://github.com/desihub/desispec/blob/master/py/desispec/zcatalog.py
# for fugi 2Gb file (approx 5 mins)
if os.path.isfile(file_zpix_sum_cat):
   print('zpix summary file exists - using existing copy')
else:
   print('ERROR: zpix summary file NOT FOUND')
#   print('Creating zpix summary file - this is a couple of Gb for fuji and may take 5 mins')
#   create_summary_catalog(specprod, specgroup = specgroup_type, all_columns = True, columns_list = None, output_filename = file_zpix_sum_cat)

zpix summary file exists - using existing copy


In [15]:
T_zpixsum = Table(fitsio.read(file_zpix_sum_cat)) 

In [16]:
if specprod == 'fuji':
    need_cols = ['TARGETID','SURVEY','PROGRAM','HEALPIX','TSNR2_LRG','SV_NSPEC','SV_PRIMARY','ZCAT_NSPEC','ZCAT_PRIMARY','MIN_MJD','MEAN_MJD','MAX_MJD', 'OBJTYPE'] # fuji
if specprod == 'guadalupe':
    need_cols = ['TARGETID','SURVEY','PROGRAM','HEALPIX','TSNR2_LRG','ZCAT_NSPEC','ZCAT_PRIMARY','MIN_MJD','MEAN_MJD','MAX_MJD','OBJTYPE'] # guadalupe
    # for guadalupe, iron, etc.: also add MAIN_PRIMARY and MAIN_NSPEC
    
# Replace targeting columns with updated version from zcat VAC (for Fuji), keeping BGS to find BGS_WISE targets
target_cols = ['DESI_TARGET','BGS_TARGET','SCND_TARGET','CMX_TARGET','SV1_DESI_TARGET','SV1_BGS_TARGET','SV1_SCND_TARGET',\
              'SV2_DESI_TARGET','SV2_BGS_TARGET','SV2_SCND_TARGET','SV3_DESI_TARGET','SV3_BGS_TARGET','SV3_SCND_TARGET']

## SJ: for faster performance, one could only read the desired columns with fitsio() from the zpix_sum file
## (not necessary for EDR, but probably for DR1 / iron)
T_zpixsum_cut = T_zpixsum[need_cols+target_cols]
T_qsom_zpixsum = join(T_qsom, T_zpixsum_cut, keys=keys_for_join, join_type='left')

In [17]:
# making sure this is the sane size as before
print(len(T_qsom_zpixsum))
if len(T_qsom) == len(T_qsom_zpixsum):
    print('Same length - all good')
else:
    print('The joined QSO maker and summary catalog df is not the same size as the QSO maker catalog')
    print('Something went wrong!')

1827985
Same length - all good


In [18]:
T_qsom_zpixsum = cut_objtype(T_qsom_zpixsum)
print(len(T_qsom_zpixsum))

1397603


In [19]:
## Adding two columns we need for the cuts
a = np.array([T_qsom_zpixsum['C_LYA'], T_qsom_zpixsum['C_CIV'], T_qsom_zpixsum['C_CIII'], \
              T_qsom_zpixsum['C_MgII'], T_qsom_zpixsum['C_Hbeta'], T_qsom_zpixsum['C_Halpha']])
T_qsom_zpixsum['QN_C_LINE_BEST'] = [max(l) for l in (a.T).tolist()]
T_qsom_zpixsum['QN_C_LINE_SECOND_BEST'] = [sorted(l)[-2] for l in (a.T).tolist()]

## 3. Join FastSpecFit

In [20]:
# Read the fastspecfit file
fsf_all = fits.open(fastspec_file)
fsf_t = Table(fsf_all['FASTSPEC'].data)
fsf_m = Table(fsf_all['METADATA'].data)

In [21]:
## SJ: could downselect to the list for Extension 2 (e.g., remove the VSHIFT and unnecessary cols)
fsf_data_cols=['TARGETID','SURVEY','PROGRAM','LOGMSTAR',\
               'CIV_1549_FLUX','CIV_1549_FLUX_IVAR', 'CIV_1549_VSHIFT','CIV_1549_SIGMA',\
               'MGII_2796_FLUX','MGII_2796_FLUX_IVAR', 'MGII_2796_VSHIFT','MGII_2796_SIGMA',\
               'MGII_2803_FLUX','MGII_2803_FLUX_IVAR', 'MGII_2803_SIGMA',\
               'OII_3726_FLUX','OII_3726_FLUX_IVAR','OII_3726_EW','OII_3726_EW_IVAR',\
               'OII_3729_FLUX','OII_3729_FLUX_IVAR','OII_3729_EW','OII_3729_EW_IVAR',\
               'NEV_3426_FLUX','NEV_3426_FLUX_IVAR','NEV_3426_VSHIFT','NEV_3426_SIGMA',\
               'HEII_4686_FLUX','HEII_4686_FLUX_IVAR',\
               'HBETA_EW','HBETA_EW_IVAR','HBETA_FLUX','HBETA_FLUX_IVAR','HBETA_VSHIFT','HBETA_SIGMA',\
               'HBETA_BROAD_FLUX', 'HBETA_BROAD_FLUX_IVAR', 'HBETA_BROAD_VSHIFT','HBETA_BROAD_SIGMA','HBETA_BROAD_CHI2',\
               'OIII_5007_FLUX','OIII_5007_FLUX_IVAR','OIII_5007_VSHIFT','OIII_5007_SIGMA',\
               'OI_6300_FLUX','OI_6300_FLUX_IVAR','OI_6300_VSHIFT','OI_6300_SIGMA',\
               'HALPHA_EW', 'HALPHA_FLUX','HALPHA_FLUX_IVAR','HALPHA_VSHIFT','HALPHA_SIGMA', \
               'HALPHA_BROAD_FLUX','HALPHA_BROAD_FLUX_IVAR','HALPHA_BROAD_VSHIFT','HALPHA_BROAD_SIGMA',\
               'NII_6584_FLUX','NII_6584_FLUX_IVAR','NII_6584_VSHIFT','NII_6584_SIGMA',\
               'SII_6716_FLUX','SII_6716_FLUX_IVAR','SII_6716_VSHIFT','SII_6716_SIGMA',\
               'SII_6731_FLUX','SII_6731_FLUX_IVAR','SII_6731_VSHIFT','SII_6731_SIGMA']

fsf_meta_cols=['TARGETID','SURVEY','PROGRAM','PHOTSYS','LS_ID',\
               'FIBERFLUX_G','FIBERFLUX_R','FIBERFLUX_Z','FIBERTOTFLUX_G','FIBERTOTFLUX_R','FIBERTOTFLUX_Z',\
               'FLUX_G','FLUX_R','FLUX_Z','FLUX_W1','FLUX_W2','FLUX_W3','FLUX_W4',\
               'FLUX_IVAR_G','FLUX_IVAR_R','FLUX_IVAR_Z','FLUX_IVAR_W1','FLUX_IVAR_W2','FLUX_IVAR_W3','FLUX_IVAR_W4',\
               'EBV','MW_TRANSMISSION_G','MW_TRANSMISSION_R','MW_TRANSMISSION_Z',\
               'MW_TRANSMISSION_W1','MW_TRANSMISSION_W2','MW_TRANSMISSION_W3','MW_TRANSMISSION_W4']

In [22]:
fsf_t_cut = fsf_t[fsf_data_cols]
fsf_m_cut = fsf_m[fsf_meta_cols]

In [23]:
T_fsf_cut = join(fsf_m_cut, fsf_t_cut, join_type='left', keys=keys_for_join)
T_fsf = join(T_qsom_zpixsum, T_fsf_cut, join_type='left', keys=keys_for_join)

In [24]:
T = T_fsf
print(len(T))

1397603


In [25]:
print(len(T_fsf_cut))

1397603


In [26]:
T_fsf.columns

<TableColumns names=('TARGETID','Z','ZERR','ZWARN','SPECTYPE','COADD_FIBERSTATUS','TARGET_RA','TARGET_DEC','MORPHTYPE','MASKBITS','COADD_NUMEXP','COADD_EXPTIME','TSNR2_LYA','TSNR2_QSO','DELTA_CHI2_MGII','A_MGII','SIGMA_MGII','B_MGII','VAR_A_MGII','VAR_SIGMA_MGII','VAR_B_MGII','Z_RR','Z_QN','C_LYA','C_CIV','C_CIII','C_MgII','C_Hbeta','C_Halpha','Z_LYA','Z_CIV','Z_CIII','Z_MgII','Z_Hbeta','Z_Halpha','QSO_MASKBITS','SURVEY','PROGRAM','HEALPIX','TSNR2_LRG','SV_NSPEC','SV_PRIMARY','ZCAT_NSPEC','ZCAT_PRIMARY','MIN_MJD','MEAN_MJD','MAX_MJD','OBJTYPE','DESI_TARGET','BGS_TARGET','SCND_TARGET','CMX_TARGET','SV1_DESI_TARGET','SV1_BGS_TARGET','SV1_SCND_TARGET','SV2_DESI_TARGET','SV2_BGS_TARGET','SV2_SCND_TARGET','SV3_DESI_TARGET','SV3_BGS_TARGET','SV3_SCND_TARGET','QN_C_LINE_BEST','QN_C_LINE_SECOND_BEST','PHOTSYS','LS_ID','FIBERFLUX_G','FIBERFLUX_R','FIBERFLUX_Z','FIBERTOTFLUX_G','FIBERTOTFLUX_R','FIBERTOTFLUX_Z','FLUX_G','FLUX_R','FLUX_Z','FLUX_W1','FLUX_W2','FLUX_W3','FLUX_W4','FLUX_IVAR_G','FLU

## 5. Read Yaml

In [27]:
import importlib
importlib.reload(set_agn_masksDESI)
from set_agn_masksDESI import get_agn_maskbits
from set_agn_masksDESI import update_AGN_MASKBITS

AGN_MASKBITS, OPT_UV_TYPE, IR_TYPE = get_agn_maskbits('../py/agnmask.yaml')

In [28]:
AGN_MASKBITS

AGN_MASKBITS:
  - [AGN_ANY,          0, "any AGN classification is set"]
  - [RR,               1, "RR determines this to be a QSO from template fitting"]
  - [MGII,             2, "MgII afterburner detects broad line"]
  - [QN,               3, "Quasar Net reclassifies as a QSO"]
  - [QN_NEW_RR,        4, "Quasar Net prompts different RR redshift"]
  - [QN_BGS,           5, "Quasar Net reclassifies BGS target as a QSO"]
  - [QN_ELG,           6, "Quasar Net reclassifies ELG target as a QSO"]
  - [QN_VAR_WISE,      7, "Quasar Net reclassifies VAR_WISE_QSO target as a QSO"]
  - [BPT_ANY_SY,      10, "At least one BPT diagnostic indicates SEYFERT (robust AGN)"]
  - [BPT_ANY_AGN,     11, "At least one BPT diagnostic indicates SEYFERT, LINER or COMPOSITE"]
  - [BROAD_LINE,      12, "Lines with FWHN >=1200 km/s in Halpha, Hbeta, MgII and/or CIV line"]
  - [OPT_OTHER_AGN,   13, "Rest frame optical emission lines diagnostic not BPT (4000-10000 ang) indicate AGN"]
  - [UV,              14, "Re

In [29]:
OPT_UV_TYPE

OPT_UV_TYPE:
  - [NII_BPT,          0, "NII BPT diagnostic is avalible (update_AGNTYPE_NIIBPT)"]
  - [NII_SF,           1, "NII BPT Star-forming (update_AGNTYPE_NIIBPT)"]
  - [NII_COMP,         2, "NII BPT Composite (update_AGNTYPE_NIIBPT)"]
  - [NII_SY,           3, "NII BPT Seyfert (update_AGNTYPE_NIIBPT)"]
  - [NII_LINER,        4, "NII BPT LINER (update_AGNTYPE_NIIBPT)"]
  - [SII_BPT,          5, "SII BPT diagnostic is available (update_AGNTYPE_SIIBPT)"]
  - [SII_SF,           6, "SII BPT Star-forming (update_AGNTYPE_SIIBPT)"]
  - [SII_SY,           7, "SII BPT Seyfert (update_AGNTYPE_SIIBPT)"]
  - [SII_LINER,        8, "SII BPT LINER (update_AGNTYPE_SIIBPT)"]
  - [OI_BPT,           9, "OI BPT diagnostic is available (update_AGNTYPE_OIBPT)"]
  - [OI_SF,           10, "OI BPT Star-forming (update_AGNTYPE_OIBPT)"]
  - [OI_SY,           11, "OI BPT Seyfert (update_AGNTYPE_OIBPT)"]
  - [OI_LINER,        12, "OI BPT LINER (update_AGNTYPE_OIBPT)"]
  - [WHAN,            13, "WHAN is avail

In [30]:
IR_TYPE

IR_TYPE:
  - [WISE_W12,         0, "WISE W1 and W2 available (update_AGNTYPE_WISE_colors)"]
  - [WISE_W123,        1, "WISE W1, W2 and W3 available"]
  - [WISE_AGN_J11,     2, "WISE diagnostic Jarrett et al. 2011 is AGN (based on W1,W2,W3)"]
  - [WISE_SF_J11,      3, "WISE diagnostic Jarrett et al. 2011 is not an AGN (based on W1,W2,W3)"]
  - [WISE_AGN_S12,     4, "WISE diagnostic Stern et al. 2012 is AGN (based on W1,W2)"]
  - [WISE_SF_S12,      5, "WISE diagnostic Stern et al. 2012 is not an AGN (based on W1,W2)"]
  - [WISE_AGN_M12,     6, "WISE diagnostic Mateos et al. 2012 is AGN (based on W1,W2,W3)"]
  - [WISE_SF_M12,      7, "WISE diagnostic Mateos et al. 2012 is not an AGN (based on W1,W2,W3)"]
  - [WISE_AGN_A18,     8, "WISE diagnostic Assef et al. 2018 is AGN (based on W1,W2)"]
  - [WISE_SF_A18,      9, "WISE diagnostic Assef et al. 2018 is not an AGN (based on W1,W2)"]
  - [WISE_AGN_Y20,    10, "WISE diagnostic Yao et al. 2020 is AGN (based on W1,W2,W3)"]
  - [WISE_SF_Y20,   

## 6. Set QSO_MASKBITS part of AGN_MASKBITS

In [31]:
T = update_AGN_MASKBITS(T, AGN_MASKBITS, snr=3, snrOI=1, Kewley01=False, mask=None)

In [32]:
T.columns

<TableColumns names=('TARGETID','Z','ZERR','ZWARN','SPECTYPE','COADD_FIBERSTATUS','TARGET_RA','TARGET_DEC','MORPHTYPE','MASKBITS','COADD_NUMEXP','COADD_EXPTIME','TSNR2_LYA','TSNR2_QSO','DELTA_CHI2_MGII','A_MGII','SIGMA_MGII','B_MGII','VAR_A_MGII','VAR_SIGMA_MGII','VAR_B_MGII','Z_RR','Z_QN','C_LYA','C_CIV','C_CIII','C_MgII','C_Hbeta','C_Halpha','Z_LYA','Z_CIV','Z_CIII','Z_MgII','Z_Hbeta','Z_Halpha','QSO_MASKBITS','SURVEY','PROGRAM','HEALPIX','TSNR2_LRG','SV_NSPEC','SV_PRIMARY','ZCAT_NSPEC','ZCAT_PRIMARY','MIN_MJD','MEAN_MJD','MAX_MJD','OBJTYPE','DESI_TARGET','BGS_TARGET','SCND_TARGET','CMX_TARGET','SV1_DESI_TARGET','SV1_BGS_TARGET','SV1_SCND_TARGET','SV2_DESI_TARGET','SV2_BGS_TARGET','SV2_SCND_TARGET','SV3_DESI_TARGET','SV3_BGS_TARGET','SV3_SCND_TARGET','QN_C_LINE_BEST','QN_C_LINE_SECOND_BEST','PHOTSYS','LS_ID','FIBERFLUX_G','FIBERFLUX_R','FIBERFLUX_Z','FIBERTOTFLUX_G','FIBERTOTFLUX_R','FIBERTOTFLUX_Z','FLUX_G','FLUX_R','FLUX_Z','FLUX_W1','FLUX_W2','FLUX_W3','FLUX_W4','FLUX_IVAR_G','FLU

In [33]:
## Sanity check: are there several different values for the new AGN_MASKBITS column?
print(np.unique(T['AGN_MASKBITS']))

AGN_MASKBITS
------------
           0
           2
           4
           6
          10
          14
          24
          26
          28
          30
         ...
       38976
       39936
       39938
       39940
       39942
       39946
       39950
       39960
       39964
       39968
       40000
Length = 109 rows


## 7. Set diagnostic bits 

In [48]:
from set_agn_masksDESI import *

In [49]:
## OPT_UV_TYPE
T = update_AGNTYPE_NIIBPT(T, OPT_UV_TYPE, snr=3, mask=None)
T = update_AGNTYPE_SIIBPT(T, OPT_UV_TYPE, snr=3, Kewley01=False, mask=None)
T = update_AGNTYPE_OIBPT(T, OPT_UV_TYPE, snr=3, snrOI=1, Kewley01=False, mask=None)
T = update_AGNTYPE_WHAN(T, OPT_UV_TYPE, snr=3, mask=None)
T = update_AGNTYPE_BLUE(T, OPT_UV_TYPE, snr=3, snrOII=3, mask=None)
T = update_AGNTYPE_MEX(T, OPT_UV_TYPE, snr=3, mask=None)
T = update_AGNTYPE_KEX(T, OPT_UV_TYPE, snr=3, mask=None)
T = update_AGNTYPE_HeII(T, OPT_UV_TYPE, snr=3, mask=None)
T = update_AGNTYPE_NeV(T, OPT_UV_TYPE, snr=2.5, mask=None)
## IR_TYPE
T = update_AGNTYPE_WISE_colors(T, IR_TYPE, snr=3, mask=None)

In [50]:
print(np.unique(T['OPT_UV_TYPE']))

OPT_UV_TYPE 
------------
       16384
       24576
       32768
       40960
       65536
       73728
      131072
      139264
      262144
      270336
         ...
374518362917
391481157219
391488497251
391496885859
391548265987
391548266083
391555606115
391557703267
391563994627
391563994723
391689823843
Length = 3895 rows


In [55]:
## Example case to check that some number are set as [NII]-BPT LINERs
is_nii_liner = (T['OPT_UV_TYPE'] & OPT_UV_TYPE.NII_SF !=0)
len(T[is_nii_liner])

155350

In [56]:
T.columns

<TableColumns names=('TARGETID','Z','ZERR','ZWARN','SPECTYPE','COADD_FIBERSTATUS','TARGET_RA','TARGET_DEC','MORPHTYPE','MASKBITS','COADD_NUMEXP','COADD_EXPTIME','TSNR2_LYA','TSNR2_QSO','DELTA_CHI2_MGII','A_MGII','SIGMA_MGII','B_MGII','VAR_A_MGII','VAR_SIGMA_MGII','VAR_B_MGII','Z_RR','Z_QN','C_LYA','C_CIV','C_CIII','C_MgII','C_Hbeta','C_Halpha','Z_LYA','Z_CIV','Z_CIII','Z_MgII','Z_Hbeta','Z_Halpha','QSO_MASKBITS','SURVEY','PROGRAM','HEALPIX','TSNR2_LRG','SV_NSPEC','SV_PRIMARY','ZCAT_NSPEC','ZCAT_PRIMARY','MIN_MJD','MEAN_MJD','MAX_MJD','OBJTYPE','DESI_TARGET','BGS_TARGET','SCND_TARGET','CMX_TARGET','SV1_DESI_TARGET','SV1_BGS_TARGET','SV1_SCND_TARGET','SV2_DESI_TARGET','SV2_BGS_TARGET','SV2_SCND_TARGET','SV3_DESI_TARGET','SV3_BGS_TARGET','SV3_SCND_TARGET','QN_C_LINE_BEST','QN_C_LINE_SECOND_BEST','PHOTSYS','LS_ID','FIBERFLUX_G','FIBERFLUX_R','FIBERFLUX_Z','FIBERTOTFLUX_G','FIBERTOTFLUX_R','FIBERTOTFLUX_Z','FLUX_G','FLUX_R','FLUX_Z','FLUX_W1','FLUX_W2','FLUX_W3','FLUX_W4','FLUX_IVAR_G','FLU

## 8. Join multiwavelength surveys

### NOTE: this should be done OUTSIDE of this workflow and could be run once on all targets (observed and unobserved)

In [39]:
# sdss, X-ray, IR

## 9. Save catalog

In [57]:
T_final_cols = T[final_cols]
T_sec_ext = T[ext2_cols]

In [41]:
%%time
## Write output file to a temporary file;
## Rename to a version number once ready to freeze a new version
#T.write(dir_for_cat+'agnqso_sum_devBCtest.fits', format='fits', overwrite=True)
#T_final_cols.write(dir_for_cat+'agnqso_sum_vBCtest.fits', format='fits', overwrite=True)

CPU times: user 2 µs, sys: 1 µs, total: 3 µs
Wall time: 4.77 µs


In [42]:
#hdu = fits.PrimaryHDU(data=data)

In [58]:
%%time

## This will become the official catalog: "agnqso_sum_v1.x.fits" with an updated Version number
primary_hdu = fits.PrimaryHDU()
table1_hdu = fits.BinTableHDU(T_final_cols)
table2_hdu = fits.BinTableHDU(T_sec_ext)
hdulist = fits.HDUList([primary_hdu, table1_hdu, table2_hdu])
hdulist.writeto(dir_for_cat+'agnqso_sum_v1.7.fits', overwrite=False)

CPU times: user 12.2 s, sys: 1.52 s, total: 13.7 s
Wall time: 14 s
